In [1]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
from backtesting.test import SMA
import pandas as pd
from bokeh.io import export_png
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import Position

/Users/fahad/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:34: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
PFE_predictions = pd.read_csv("/Users/fahad/project_repo/models/final_models/Financial_Analysis/predictions/PFE_predictions.csv", index_col=0)


In [3]:
PFE_predictions.head()

,Date,Predictions
0,2019-07-31,0
1,2019-06-28,0
2,2019-05-31,0
3,2019-04-30,1
4,2019-03-29,1


In [4]:
PFE_predictions.Date = pd.to_datetime(PFE_predictions.Date, format="%Y-%m-%d")

In [5]:
PFE_predictions['Predictions'] = PFE_predictions['Predictions'].shift(-1)


In [6]:
PFE_predictions = PFE_predictions.dropna()


In [7]:
PFE_predictions = PFE_predictions[::-1]

In [8]:
PFE_predictions.head(10)

,Date,Predictions
112,2010-03-31,0.0
111,2010-04-30,1.0
110,2010-05-28,1.0
109,2010-06-30,1.0
108,2010-07-30,1.0
107,2010-08-31,1.0
106,2010-09-30,1.0
105,2010-10-29,1.0
104,2010-11-30,1.0
103,2010-12-31,1.0


In [9]:
PFE_monthly = pd.read_csv("/Users/fahad/project_repo/data/final/PFE/PFE_monthly.csv", index_col =0)

In [10]:
PFE_monthly.columns = ['Open','High','Low','Close','Volume']

In [11]:
PFE_predictions = PFE_predictions.set_index("Date")

In [12]:
PFE_monthly.head()

,Open,High,Low,Close,Volume
2019-08-15,38.3100,38.620,33.97,34.48,377359300
2019-07-31,43.5100,44.557,38.51,38.84,560639900
2019-06-28,41.6334,44.290,41.57,43.32,437559700
2019-05-31,40.5900,42.480,39.87,41.52,524538400
2019-04-30,42.5800,43.280,38.42,40.61,461322900


In [13]:
PFE_final_df = pd.merge(PFE_predictions, PFE_monthly, right_index=True, left_index=True)

In [14]:
PFE_final_df.index = pd.to_datetime(PFE_final_df.index, format ="%Y-%m-%d")

In [15]:
equity_dict = dict()

In [16]:
class Binary_Classification_1d(TrailingStrategy):

    
    def init(self):
        
        super().init()
        self.data.Predictions
        self.set_trailing_sl(4)
        
    def next(self):
        super().init()
        equity_dict[self.data.index[-1]] = TrailingStrategy.equity.__get__(self)
        
        position = Strategy.position.__get__(self)
        
        if self.data.Predictions[-1] == 1.0  and (position.is_long!=True):
#             print("\n")
#             print (self.data.Prediction[-1])
            
            self.buy()

        # Else, if sma1 crosses below sma2, sell it
        elif self.data.Predictions[-1] == 0.0  and (position.is_short!=True):
#             print("\n")
#             print (self.data.Prediction[-1])
            
            self.sell()
            
 

In [17]:
bt = Backtest(PFE_final_df, Binary_Classification_1d, cash=10000, commission=.01)
bt.run()



Start                          2010-03-31 00:00:00
End                            2019-07-31 00:00:00
Duration                        3409 days 00:00:00
Exposure [%]                               98.2986
Equity Final [$]                            128643
Equity Peak [$]                             128643
Return [%]                                 1186.43
Buy & Hold Return [%]                      126.472
Max. Drawdown [%]                         -15.9595
Avg. Drawdown [%]                         -3.88552
Max. Drawdown Duration           273 days 00:00:00
Avg. Drawdown Duration            97 days 00:00:00
# Trades                                        51
Win Rate [%]                               76.4706
Best Trade [%]                             23.1911
Worst Trade [%]                           -4.20631
Avg. Trade [%]                             4.99326
Max. Trade Duration              427 days 00:00:00
Avg. Trade Duration               66 days 00:00:00
Expectancy [%]                 

In [18]:
bt.plot(superimpose=False, plot_volume=False)

In [19]:
df = pd.DataFrame.from_dict([equity_dict])

In [20]:
df = df.T

In [21]:
df.columns = ['Equity']

In [22]:
export = df.to_csv("/Users/fahad/project_repo/data/final/Equity/PFE_CNN_Equity.csv")

In [23]:
df

,Equity
2010-04-30,10000.000000
2010-05-28,8975.719000
2010-06-30,8404.054691
2010-07-30,8840.169731
2010-08-31,9376.473362
2010-09-30,10119.047619
2010-10-29,10266.383781
2010-11-30,9606.317775
2010-12-31,10319.424800
2011-01-31,10737.859500
